# 6.7: Tree-Based Methods

In [ ]:
# Import libraries and specific objects
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table
from ISLP.models import (ModelSpec as MS,
                         summarize)
from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestRegressor as RF,
      GradientBoostingRegressor as GBR)
from ISLP.bart import BART

import warnings
# Suppress FutureWarning in ISLP.models.columns
# The warning is related to Series.__getitem__ treating keys as positions, which is deprecated.
# Since ISLP is an external library that I don't control, and this specific warning does not
# affect my current usage, I'm suppressing it to keep the output clean and focused on relevant information.
warnings.filterwarnings(action='ignore', category=FutureWarning, module='ISLP.models.columns')

### Fitting Classification Trees

We first use classification trees to analyze the `Carseats` data set. 

In [ ]:
# Load data
Carseats = load_data('Carseats')
High = np.where(Carseats.Sales > 8,
                "Yes",
                "No")

We now use `DecisionTreeClassifier()` (imported as `DTC`) to fit a classification tree in order to predict High using all variables but Sales. To do so, we must form a model matrix as we did when fitting regression models.

In [ ]:
model = MS(Carseats.columns.drop('Sales'), intercept=False)
D = model.fit_transform(Carseats)
feature_names = list(D.columns)
X = np.asarray(D)

# Specify classifer 
clf = DTC(criterion='entropy',
          max_depth=3,
          random_state=0)        
clf.fit(X, High)

In [ ]:
ax = subplots(figsize=(12,12))[1]
plot_tree(clf,
          feature_names=feature_names,
          ax=ax);

In [ ]:
# Check the accuracy score
accuracy_score(High, clf.predict(X))

With only the default arguments, the training error rate is 21%.

Now we will try out cost complexity pruning to see if we can get a tree with a better test error.

In [ ]:
# Split data into training and test sets
(X_train,
 X_test,
 High_train,
 High_test) = skm.train_test_split(X,
                                   High,
                                   test_size=0.5,
                                   random_state=0)

In [ ]:
# We first refit the full tree on the training set
clf = DTC(criterion='entropy', random_state=0)
clf.fit(X_train, High_train)
accuracy_score(High_test, clf.predict(X_test))

In [ ]:
# Next we use the cost_complexity_pruning_path() method of clf to extract cost-complexity values
ccp_path = clf.cost_complexity_pruning_path(X_train, High_train)
kfold = skm.KFold(10,
                  random_state=1,
                  shuffle=True)

grid = skm.GridSearchCV(clf,
                        {'ccp_alpha': ccp_path.ccp_alphas},
                        refit=True,
                        cv=kfold,
                        scoring='accuracy')
grid.fit(X_train, High_train)
grid.best_score_

In [ ]:
# Count the leaves
ax = subplots(figsize=(12, 12))[1]
best_ = grid.best_estimator_
plot_tree(best_,
          feature_names=feature_names,
          ax=ax);

**Compute the test error rate of this pruned tree. How does the test error rate and the interpretability
of this tree compare to the inital tree?**

### Fitting Regression Trees

We will be fitting a regression tree to predict the median value of houses `medv` in `Boston` suburbs based on the information in the data set

In [ ]:
# Load data
Boston = load_data("Boston")
model = MS(Boston.columns.drop('medv'), intercept=False)
D = model.fit_transform(Boston)
feature_names = list(D.columns)
X = np.asarray(D)

In [ ]:
# Split into test and training
(X_train,
 X_test,
 y_train,
 y_test) = skm.train_test_split(X,
                                Boston['medv'],
                                test_size=0.3,
                                random_state=0)

In [ ]:
# Fit regression tree
reg = DTR(max_depth=3)
reg.fit(X_train, y_train)
ax = subplots(figsize=(12,12))[1]
plot_tree(reg,
          feature_names=feature_names,
          ax=ax);

In [ ]:
# Use cross-validation function to see whether pruning the tree will improve performance
ccp_path = reg.cost_complexity_pruning_path(X_train, y_train)
kfold = skm.KFold(5,
                  shuffle=True,
                  random_state=10)
grid = skm.GridSearchCV(reg,
                        {'ccp_alpha': ccp_path.ccp_alphas},
                        refit=True,
                        cv=kfold,
                        scoring='neg_mean_squared_error')
G = grid.fit(X_train, y_train)

In [ ]:
# Use pruned tree to make predictions on the test set
best_ = grid.best_estimator_
np.mean((y_test - best_.predict(X_test))**2)

The test set MSE associated with the regression tree is 28.07.

### Bagging and Random Forests

We will use bagging and random forests on the `Boston` data set, using the `RandomForestRegressor()` from the `sklearn.ensemble` package.

In [ ]:
bag_boston = RF(max_features=X_train.shape[1], random_state=0) # max_features indicates that all 12 predictors should be considered 
bag_boston.fit(X_train, y_train)

In [ ]:
ax = subplots(figsize=(8,8))[1]
y_hat_bag = bag_boston.predict(X_test)
ax.scatter(y_hat_bag, y_test)
np.mean((y_test - y_hat_bag)**2)

The test set MSE associated with the bagged regression tree is 14.63, about half that obtained using an optimally-pruned single tree. We could change the number of trees grown from the default of 100 by using the `n_estimators` argument:

In [ ]:
bag_boston = RF(max_features=X_train.shape[1],
                n_estimators=500,
                random_state=0).fit(X_train, y_train)
y_hat_bag = bag_boston.predict(X_test)
np.mean((y_test - y_hat_bag)**2)

There is not much change. Growing a random forest proceeds in exactly the same way, except that we use a smaller value of the `max_features`. Lets change `max_features=6`.

In [ ]:
RF_boston = RF(max_features=6,
               random_state=0).fit(X_train, y_train)
y_hat_RF = RF_boston.predict(X_test)
np.mean((y_test - y_hat_RF)**2)

The test set MSE is 20.04; this indicates that random forests did somewhat worse than bagging in this case. Extracting the `feature_importances_ values` from the fitted model, we can view the importance of each variable.

In [ ]:
feature_imp = pd.DataFrame(
    {'importance':RF_boston.feature_importances_},
    index=feature_names)
feature_imp.sort_values(by='importance', ascending=False)

**Which two variables are the most important when determining median house values in Boston
suburbs?**

### Boosting

Here we use `GradientBoostingRegressor()` from `sklearn.ensemble` to fit boosted regression trees to the Boston data set. For classification we would use `GradientBoostingClassifier()`. 

In [ ]:
boost_boston = GBR(n_estimators=5000,
                   learning_rate=0.001,
                   max_depth=3,
                   random_state=0)
boost_boston.fit(X_train, y_train)

In [ ]:
# Get predicted values
test_error = np.zeros_like(boost_boston.train_score_)
for idx, y_ in enumerate(boost_boston.staged_predict(X_test)):
   test_error[idx] = np.mean((y_test - y_)**2)

plot_idx = np.arange(boost_boston.train_score_.shape[0])
ax = subplots(figsize=(8,8))[1]
ax.plot(plot_idx,
        boost_boston.train_score_,
        'b',
        label='Training')
ax.plot(plot_idx,
        test_error,
        'r',
        label='Test')
ax.legend();

In [ ]:
# We now use the boosted model to predict medv on the test set

y_hat_boost = boost_boston.predict(X_test);
np.mean((y_test - y_hat_boost)**2)

The test MSE obtained is 14.48,
similar to the test MSE for bagging. If we want to, we can
perform boosting with a different value of the shrinkage parameter
$\lambda$ in  (8.10). The default value is 0.001, but
this is easily modified.  Here we take $\lambda=0.2$.

In [ ]:
boost_boston = GBR(n_estimators=5000,
                   learning_rate=0.2,
                   max_depth=3,
                   random_state=0)
boost_boston.fit(X_train,
                 y_train)
y_hat_boost = boost_boston.predict(X_test);
np.mean((y_test - y_hat_boost)**2)

In this case, using $\lambda=0.2$ leads to a almost the same test MSE
as when using $\lambda=0.001$.

**Try fitting a new boosted model to the training set using a higher value for shrinkage and
compute the test MSE. Which shrinkage parameter (between the two) yields the model with
the best test error?**

### Bayesian Additive Regression Trees

In this section we demonstrate a Python implementation of BART found in the `ISLP.bart` package. We fit a model to the Boston housing data set. This `BART()` estimator is designed for quantitative outcome variables, though other implementations are available for fitting logistic and probit models to categorical outcomes.

In [ ]:
bart_boston = BART(random_state=0, burnin=5, ndraw=15)
bart_boston.fit(X_train, y_train)

In [ ]:
# Split into test and training
yhat_test = bart_boston.predict(X_test.astype(np.float32))
np.mean((y_test - yhat_test)**2)

In [ ]:
# Check how many times each variable appeared in the collection of trees
var_inclusion = pd.Series(bart_boston.variable_inclusion_.mean(0),
                               index=D.columns)
var_inclusion

*These exercises were adapted from :* James, Gareth, et al. An Introduction to Statistical Learning: with Applications in Python, Springer, 2023.